In [1]:
import requests
import time
import json
import html
import time
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from transformers import pipeline
import sentencepiece
import pandas as pd

In [2]:
# Initialize the question generation pipeline
qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
def fetch_impulse_classes():
   
    # URL of the Wikipedia page
    url = "https://en.wikipedia.org/wiki/Model_rocket_motor_classification"
    
    # Send a request to the webpage
    response = requests.get(url)
    # print(f'response json {response.json()}')
    soup = BeautifulSoup(response.text, 'html.parser')
    # print(f'soup {soup}')
    
    # Find all tables
    tables = soup.find_all('table', {'class': 'wikitable'})
    # imp = soup.find_all('ta')
    # print(f'table {tables}')
    
    # Look for the table that contains the header "Total impulse (N·s)"
    target_table = None
    
    for table in tables:
        # print(f'table text {table.}')
        if 'impulse' in table.text:
            print('found')
            target_table = table
            break
    # print(target_table)
    # Parse the table into a DataFrame
    df = pd.read_html(str(tables))[0]
    
    # Display the resulting DataFrame
    print(df['Class  (Base 26)'])
    ls = df['Class  (Base 26)'].tolist()
    
    return [x for x in ls if len(x) < 10]


    

In [3]:
# Stack Exchange API
# def fetch_stackexchange_qa(site='space', tagged='rocket', pages=5):
#     questions = []
#     for page in range(1, pages+1):
#         url = f"https://api.stackexchange.com/2.3/questions?page={page}&pagesize=20&order=desc&sort=activity&tagged={tagged}&site={site}&filter=withbody"
#         resp = requests.get(url).json()
#         print(f'resp {resp}')
#         for item in resp.get('items', []):
#             q = {
#                 'question': item['title'],
#                 'body': item.get('body', ''),
#                 'answers': []
#             }
#             if item.get('is_answered'):
#                 a_url = f"https://api.stackexchange.com/2.3/questions/{item['question_id']}/answers?order=desc&sort=votes&site={site}&filter=withbody"
#                 a_resp = requests.get(a_url).json()
#                 for answer in a_resp.get('items', []):
#                     q['answers'].append(answer['body'])
#                     questions.append(q)
#             # questions.append(q)
#         time.sleep(1)  # Respect rate limits
#     return questions

 
# arXiv API Q&A extraction from abstract (pseudo-QA from title and abstract)
def fetch_arxiv_abstracts(query='rocket propulsion', max_results=10):
    url = f"http://export.arxiv.org/api/query?search_query=all:{query.replace(' ', '+')}&start=0&max_results={max_results}"
    resp = requests.get(url)
    root = ET.fromstring(resp.content)

    qas = []
    ns = {'atom': 'http://www.w3.org/2005/Atom'}  # arXiv uses Atom XML namespace

    for entry in root.findall('atom:entry', ns):
        title = entry.find('atom:title', ns).text.strip()
        
        summary = entry.find('atom:summary', ns).text.strip()
        # Generate question from text
        input_text = f"generate question: {summary}"
        generated_question = qg_pipeline(input_text, max_length=64, do_sample=False)[0]['generated_text']
        
        qas.append({
            'title': title,
            'question': generated_question,
            'answer': summary
        })
    return qas


In [4]:
# Wikipedia scraping

# Initialize the question generation pipeline
# qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

def clean_text(text):
    return ' '.join(text.strip().split())

def get_all_rocket_propulsion_links(base_url='https://en.wikibooks.org/wiki/Rocket_Propulsion'):
    """Grab all unique subpage links under Rocket Propulsion."""
    resp = requests.get(base_url)
    soup = BeautifulSoup(resp.text, 'html.parser')

    content_div = soup.select_one('#mw-content-text')
    links = content_div.find_all('a', href=True)
    
    urls = set()
    for link in links:
        href = link['href']
        if 'https' in href:
            full_url = href
        else:
            full_url = 'https://en.wikibooks.org' + href
            # print(f'full_url {full_url}')
        urls.add(full_url)
        # if href.startswith('/wiki/Rocket_Propulsion/') and ':' not in href:
        #     print(f'url: https://en.wikibooks.org + { href}')
        #     urls.add('https://en.wikibooks.org' + href)

    return list(urls)

def scrape_pages_with_qg(urls, visited, M, batch_size=10):
    """Scrape up to `batch_size` new pages not in `visited`, return new QAs."""
    qa_pairs = []
    count = 0
    print(f'visited {visited}')

    for url in urls:
        if url in visited:
            continue
        visited.add(url)
        try:
            resp = requests.get(url)
            soup = BeautifulSoup(resp.text, 'html.parser')
            content_div = soup.select_one('#mw-content-text')
            paragraphs = content_div.find_all('p')
            text = clean_text(' '.join(p.get_text() for p in paragraphs[:3]))
            if not text or len(text) < M:
                continue

            # Generate question from text
            input_text = f"generate question: {text}"
            output = qg_pipeline(input_text, max_length=64, do_sample=False)[0]['generated_text']

            qa_pairs.append({
                'question': output,
                'answer': text,
                'source_url': url
            })
            print(f"✅ Generated Q&A from: {url}")
            count += 1
            time.sleep(1)  # polite scraping

            if count >= batch_size:
                break
        except Exception as e:
            print(f"⚠️ Failed to process {url}: {e}")
    return qa_pairs

def scrape_wikibook_qas(M):
    all_links = get_all_rocket_propulsion_links()
    visited = set()
    all_qas = []

    for i in range(10):
        print(f"\n🔁 Batch {i+1}/10")
        batch_qas = scrape_pages_with_qg(all_links, visited, M, batch_size=10)
        all_qas.extend(batch_qas)
        if len(all_qas) >= M:
            break

    # with open('wikibook_qas_100.json', 'w', encoding='utf-8') as f:
    #     json.dump(all_qas, f, indent=2)

    print(f"\n✅ Done. Saved {len(all_qas)} Q&A pairs to wikibook_qas_100.json")
    return all_qas

In [19]:
# Stack Exchange API

def clean_html_text(html_content):
    """Remove hyperlinks and strip HTML tags from content."""
    soup = BeautifulSoup(html_content, 'html.parser')

    # Replace <a> tags with their inner text
    for a in soup.find_all('a'):
        a.replace_with(a.get_text())

    # Get cleaned text
    text = soup.get_text(separator=' ')
    return html.unescape(text.strip())

def fetch_stackexchange_qa_data(site='space.stackexchange', tag='rockets', pagesize=20, max_pages=2):
    base_url = 'https://api.stackexchange.com/2.3/questions'
    answers_url = 'https://api.stackexchange.com/2.3/questions/{ids}/answers'
    all_qas = []

    for page in range(1, max_pages + 1):
        params = {
            'site': site,
            'tagged': tag,
            'pagesize': pagesize,
            'page': page,
            'filter': 'withbody'
        }
        resp = requests.get(base_url, params=params)
        data = resp.json()

        for question in data.get('items', []):
            q_id = question['question_id']
            q_body = clean_html_text(question.get('body', ''))
            title = html.unescape(question.get('title', ''))

            # Get answers
            a_params = {
                'site': site,
                'filter': 'withbody'
            }
            a_resp = requests.get(answers_url.format(ids=q_id), params=a_params)
            answers = a_resp.json().get('items', [])

            for ans in answers:
                a_body = clean_html_text(ans.get('body', ''))
                all_qas.append({
                    'question': f"{title}\n{q_body}",
                    'answer': a_body,
                    'source_url': f"https://{site}.com/questions/{q_id}"
                })

            time.sleep(1)  # polite API usage

    return all_qas


In [20]:
if __name__ == "__main__":
    se_data = fetch_stackexchange_qa_data()
    # arxiv_data = fetch_arxiv_abstracts()
    # reddit_data = fetch_reddit_qa()
    # wikibook_data = scrape_wikibook_qas(M=100)

    # all_qas = {
    #     "stackexchange": se_data,
    #     "arxiv": arxiv_data,
    #     "reddit": reddit_data,
    #     "wikibook": wikibook_data
    # }

    # save_qas_to_file(all_qas)
    # print("Data collection complete. Saved to qa_data.json")


In [42]:
se_data

[{'question': 'Would an elongated rocket launch tube NOW be feasible and economical?\nThe general idea is to make rocket launches far more economical than they currently are... maybe even within eventual affordable range for the general populace! The goal of a tube is to significantly reduce the energy needed for the rocket to deliver its payload into orbit, as constraining the rocket\'s exhaust  should  return a lot of energy back to the rocket itself (particularly in conjunction with a sabot  Edit:  This does NOT necessarily mean a "piston"; see addendum. ). At a glance, this seems to be a more feasible (and potentially economical) alternative to other extreme solutions, such as space elevators, spin launchers, space tethers, etc., which do not seem to even be achievable with current material sciences (to say nothing of budgets). \n Consider what the optimal choice of these options must be (assuming state-of-the-art solutions applied to each of their implementations): \n \n Design a 

In [53]:
se_data[12]['question']#.split('\n')[0]

'For asteroid redirection, is “slow push” inherently more efficient than “impact”?\nProposals for asteroid redirection include direct impact as well as a long thrust (either after a soft landing or by gravity tug). Purely in terms of converting a given amount of energy (chemical rocket or solar powered ion rocket) into asteroid delta-v, is one strategy inherently more efficient than the other ? \n Complications: \n \n The two strategies have different intercept strategies. The soft landing  would require a rendezvous and therefore likely require  more delta-v for the spacecraft. \n \n For a NEA interception, the optimum interception would likely be at a different location in the asteroid\'s orbit for the two methods, with different delta-v requirements for the spacecrafts \n \n A rotating asteroid complicates a soft landing and could require propellant expenditure to change asteroid angular momentum, depending on latitude of the landing site. A soft landing at one of the poles would no

In [52]:
se_data[12]['answer']

'Landing and then burning an ion engine (e.g. soft landing followed by a long thrust) would likely be more efficient. Ion engines\' efficiency means that any sort of "payload" mass added to a direct impactor to increase the delta-v it generates would better be put into an extra ion engine and fuel for it. \n Consider a two-ton spacecraft: one ton of controls, RCS, antennae, solar panels, etc. and one ton to do whatever we want with. If you send it on a trajectory to go rendezvous with an asteroid at 7 km/s, you can either \n \n load the extra ton with tungsten and deliver an extra  $7\\,000\\,000 \\text{ kg m/s}$  of momentum, or \n load an  NSTAR ion engine, with a specific impulse of $3\\,300\\text{ s}$ and a mass of $8.33\\text{ kg}$ , into the spare slot. Assuming a 10:1 mass fraction for the remaining mass, you get a dry mass for the spacecraft of roughly 1010 kilograms and a wet mass of 2 tons, netting you around 990 kilograms of fuel, so at 3300 newton-seconds of impulse per new

In [18]:
se_data[1]["body"].replace("<p>","").replace('"','').replace("</blockquote>"," ").replace("<a>","").replace("<blockquote>"," ")

"Roscosmos <a href=https://arstechnica.com/science/2022/07/russia-says-its-space-station-partnership-will-end-after-two-more-years/ rel=noreferrer>has announced their ISS partnership will end in 2 years</a></p>\n \nThis might have been seen as an indication that the new administration at Roscosmos was in a more cooperative mood. Any such hopes were dashed on Tuesday, when Borisov announced that Russia would not be renewing its current commitment to the ISS, which ends in 2024. NASA's current plans involve keeping the station occupied through the end of the decade.</p>\n \nIt's not clear what their plans are, but the new head of Roscosmos did say this</p>\n \nAccording to The New York Times, Borisov told Russian President Vladimir Putin that the 2024 date gives his country time as well. “I think that by this time, we will begin to form the Russian orbital station,” he said.</p>\n \nIn theory, Russia may want some of its ISS segments back for this &quot;new&quot; Russian space station. T